In [1]:
import time
import pyspark
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel

## Apply power_map function to all the numbers in Data/numbers.txt using .map()

In [2]:
class Power:
    def __init__(self, p):
        self.p = p
        time.sleep(2)
        
    def applyPower(self, x):
        return x**self.p
        
# map
def power_map(num):
    # apply num ** 5
    c = Power(5)
    return c.applyPower(num)

In [3]:
conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')
numbers = sc.textFile("../Data/numbers.txt", 5).map(lambda x : int(x))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/01 14:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/01 14:20:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/01 14:20:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/01 14:20:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/01 14:20:38 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Without Persisting or Caching

In [5]:
start = time.time()
powered_num = numbers.map(power_map)
powered_num.collect()
print("first ", time.time() - start)

start = time.time()
powered_num.collect()
print("second ", time.time() - start)

first  43.47693586349487


second  42.28306317329407


## With Persisting/Caching

In [6]:
powered_num.cache()

PythonRDD[2] at collect at /var/folders/0q/77jvn_416ybbg7m9_c4f7xkw2wwy7f/T/ipykernel_77278/705686008.py:3

In [7]:
start = time.time()
powered_num.collect()
print("first ", time.time() - start)  # trigger re-evaluation

first  42.24815106391907


In [8]:
start = time.time()
powered_num.collect()
print("second ", time.time() - start)  # doesn't trigger re-evaluation

second  0.1414949893951416


## What happens?

In [9]:
powered_num.persist(StorageLevel.MEMORY_AND_DISK)

Py4JJavaError: An error occurred while calling o30.persist.
: java.lang.UnsupportedOperationException: Cannot change storage level of an RDD after it was already assigned a level
	at org.apache.spark.errors.SparkCoreErrors$.cannotChangeStorageLevelError(SparkCoreErrors.scala:111)
	at org.apache.spark.rdd.RDD.persist(RDD.scala:168)
	at org.apache.spark.rdd.RDD.persist(RDD.scala:192)
	at org.apache.spark.api.java.JavaRDD.persist(JavaRDD.scala:51)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands

## How to solve it?

In [10]:
powered_num.unpersist()

PythonRDD[2] at collect at /var/folders/0q/77jvn_416ybbg7m9_c4f7xkw2wwy7f/T/ipykernel_77278/705686008.py:3

## Reading/writing from disk

In [11]:
powered_num.persist(StorageLevel.DISK_ONLY)
start = time.time()
powered_num.collect()
print("first ", time.time() - start)  # trigger reevaluation

start = time.time()
powered_num.collect()
print("second ", time.time() - start)  # doesn't trigger reevaluation

first  42.283362865448
second  0.13137602806091309


In [ ]:
powered_num.unpersist()

In [ ]:
sc.stop()